## Deepseek R1 qwen3-8b

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd

c:\Users\franc\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


the model used

In [2]:
model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B-bnb-4bit"
device = "cuda" 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-0528-Qwen3-8B")
model.to(device)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096, padding_idx=151654)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,),

an example of one translation

In [3]:
prompt = "Marco Cornelio ch'era de' dieci compagni, studiosamente  si riservò di parlare all'ultimo."

messages = [
        {"role": "system", "content": "you are a translator from old italian to model italian. you take a sentence in old italian and you answer only with: La traduzione è:<translation> . Don't add anything else. Translate only in italian"},
        {"role": "system", "content": "only use italian and no other language in the translation"},
        {"role": "user", "content": prompt} 
    ]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=5000,
    do_sample=True, 
    temperature=0.7,
    top_p=0.9,
)


new_tokens = generated_ids[0][len(model_inputs[0]):]
decoded = tokenizer.decode(new_tokens, skip_special_tokens=True)
decoded = decoded.split("La traduzione è:")[-1].strip()
print(decoded)

Marco Cornelio, uno dei dieci compagni, si riservò attentamente di parlare solo all'ultimo.


the code to translate all the database

In [ ]:
# Read the CSV file
df = pd.read_csv('dataset_cleaned.csv')

# Create new column for translations
df['Deepseek R1 qwen-8b'] = ''

# Process each row
for idx, row in df.iterrows():
    # Create messages with current prompt
    messages = [
        {"role": "system", "content": "you are a translator from old italian to model italian. you take a sentence in old italian and you answer only with: La traduzione è:<translation> . Don't add anything else. Translate only in italian"},
        {"role": "system", "content": "only use italian and no other language in the translation"},
        {"role": "user", "content": row['Sentence']} 
    ]
    
    # Prepare input
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate translation
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=10000,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    # Decode and clean up response
    new_tokens = generated_ids[0][len(model_inputs[0]):]
    decoded = tokenizer.decode(new_tokens, skip_special_tokens=True)
    translation = decoded.split("La traduzione è:")[-1].strip()    
    print(translation)
    
    # Store translation
    df.at[idx, 'Deepseek R1 qwen-8b'] = translation

# Save updated dataframe to dataset_deepseek.csv
df.to_csv('./dataset/dataset_deepseek.csv', index=False)


In quale battaglia, certamente io avevo sempre il coraggio di discutere di pace e mi sembrava che non soltanto la pace fosse disprezzata.
L'abitudine seconda spesso può essere spaventosa a causa della sua grande provvidenza, e l'abitudine prima può essere audace al punto di far impazzire.
Andò nel campo degli Cartaginesi e tutta la legione lo seguì in silenzio.
Non lo volle riconoscere come nemico. Questa è quella che diede coraggio al profeta Nathan per riprendere con grande autorità quel re, che aveva peccato.
Le armi e insieme con loro passarono tra i nemici, quindi se nessuno non avesse avuto il coraggio di questo, e quindi avevano quell'atteggiamento.
Perché ragionevolmente si interpreta Iob come afflitto;
Chi, tra tutti, perdonasse di più ai cittadini, e a cui foste più sicuri di credere, poi che egli era vostro comandante.
Chi ha scritto questo libro deve essere creduto fedelmente lo Spirito Santo, e chiedersi chi l'abbia scritto è inutile.
se non fosse così usata, ciò apparireb

cleaning

In [3]:
#clean the vram memory
torch.cuda.empty_cache()
del model
del tokenizer

## Mistral Mistral-7B-Instruct-v0.3

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import pandas as pd

c:\Users\franc\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


the model used

In [3]:

device = "cuda" 
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model.to(device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:39<00:00, 13.07s/it]


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

one translation

In [5]:
prompt = "la moltitudine de' quali tu ài potuto vedere e riguardare lo studio e poco dinanzi udire le voci, e lle cui mani e lance apena posso ritenere."

messages = [
        {"role": "system", "content": "Sei un traduttore dall'italiano antico all'italiano moderno. Traduci una frase in italiano moderno e rispondi solo con: La traduzione è:<traduzione>. Non aggiungere altro pena la morte. Usa solo l'italiano e nessun'altra lingua nella traduzione."},
        {"role": "user", "content": prompt} 
    ]

tokens = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
attention_mask = torch.ones_like(tokens)

# Move to device
model_inputs1 = tokens.to(device)
attention_mask = attention_mask.to(device)

# Generate with attention mask
generated_ids1 = model.generate(
    model_inputs1, 
    attention_mask=attention_mask,
    max_new_tokens=1000, 
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,
    top_p=0.9
)

# Decode only the new tokens (exclude the input)
new_tokens = generated_ids1[0][len(tokens[0]):]
decoded = tokenizer.decode(new_tokens, skip_special_tokens=True)
print(decoded)


La traduzione è: "La folla di coloro che hai potuto vedere e guardare lo studio e di quelli che stavano poco lontano e udire le voci, e le mani e le armi di pochi che posso ancora ricordare." (Traduction: "The crowd of those whom you have been able to see and observe in the study, and of those who were just a little away and hear the voices, and the hands and weapons of a few that I can still remember.")


code for all database

In [ ]:
df = pd.read_csv('dataset_cleaned.csv')
df['mistral'] = ''

# Process each row
for idx, row in df.iterrows():
    # Create messages with current prompt
    messages = [
        {"role": "system", "content": "Sei un traduttore dall'italiano antico all'italiano moderno. Traduci una frase in italiano moderno e rispondi solo con: La traduzione è:<traduzione>. Non aggiungere altro pena la morte. Usa solo l'italiano e nessun'altra lingua nella traduzione."},
        {"role": "user", "content": row['Sentence']} 
    ]

    tokens = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
    attention_mask = torch.ones_like(tokens)

    # Move to device
    model_inputs1 = tokens.to(device)
    attention_mask = attention_mask.to(device)

    # Generate with attention mask
    generated_ids1 = model.generate(
        model_inputs1, 
        attention_mask=attention_mask,
        max_new_tokens=1000, 
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,
        top_p=0.9
    )
    new_tokens = generated_ids1[0][len(tokens[0]):]
    decoded = tokenizer.decode(new_tokens, skip_special_tokens=True)
    translation = decoded.split("La traduzione è:")[-1].strip()
    #remove any part between ( and  )
    translation = translation.split('(')[0].strip()
    #remove anything after the \n character
    translation = translation.split('\n')[0].strip() 
    print(translation)
    
    # Store translation
    df.at[idx, 'Mistral 7b-instruction'] = translation

# Save updated dataframe to dataset_deepseek.csv
df.to_csv('./dataset/dataset_mistral.csv', index=False)


Quella guerra fu bene condotta, ma da una parte, Aiace era un cavaliere francese e valente in battaglia, ma non era molto sagace.
"Crudele, e di tutte le colpe prende vendetta, come dice la legge, e a nessun cavaliere perdonerai peccati."
"Ponzio Aufidiano, cavaliere romano, non fu arricchito di altra forza d'animo."
Se questo piace a tutti e se il tempo ha bisogno di avere Pompeio come cavaliere e non come compagno, io non lo riterrò ancora come destinato.
Sembra che questa arte faccia dire appositamente per far credere, il fine è far credere per quanto si dice.
"Quarantaquattro grandi ventagli cadono delle nebbie decisive; e potresti credere che tutto il cielo cadesse nel mare."
"Ma chi potrebbe sperare di quello che ancora questi non credono in Cristo, lo possono vedere con noi, e perché non possono negare, lo fanno con denti e grida."
Le vendite dei morti e le pressioni dei vivi fecero il tranello di un re ferocio.
Affinché quelli, il quale ora per le sue grandi reati è ferocissimo

In [23]:
#clean the vram memory
torch.cuda.empty_cache()
del model
del tokenizer

## Prometeus

In [22]:
from prometheus_eval.prompts import ABSOLUTE_PROMPT, SCORE_RUBRIC_TEMPLATE
from transformers import AutoModelForCausalLM, AutoTokenizer
import re
import pandas
from tqdm import tqdm

In [23]:
device = 'cuda'
model = AutoModelForCausalLM.from_pretrained("Unbabel/M-Prometheus-3B")
tokenizer = AutoTokenizer.from_pretrained("Unbabel/M-Prometheus-3B")

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


In [ ]:
dataset_model_mistral = pandas.read_csv('./dataset/dataset_mistral_full.csv')
dataset_model_deepseek = pandas.read_csv('./dataset/dataset_deepseek_full.csv')
dataset_golden = pandas.read_csv('./dataset/dataset_goldenLabel.csv')
dataset_model_mistral

,Author,Date,Region,Sentence,Mistral 7b-instruction
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...,"Quella guerra fu bene condotta, ma da una part..."
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c...","""Crudele, e di tutte le colpe prende vendetta,..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...,"""Ponzio Aufidiano, cavaliere romano, non fu ar..."
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...,Se questo piace a tutti e se il tempo ha bisog...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...,Sembra che questa arte faccia dire appositamen...
...,...,...,...,...,...
92,Grazia di Meo,1343,tosc.,"Alchuno è riccho e gentile, ma lamentasi che v...","Alcuni sono ricchi e gentili, ma lamentano sic..."
93,Zanobi da Strada,1361,tosc.,"Pietro, essendogli mostrato in figura il popul...","Pietro, mostrato il popolo Gentile in figura, ..."
94,Declamazioni di Seneca,1392,tosc.,pregollo che lo liberasse di quella obbligazio...,"il gentiluomo lo fece uscire dal suo impegno, ..."
95,Giovanni da San Miniato,1415,tosc.,L'oro verrà dall'Aquilone. Che figuriamo noi p...,L'oro arriverà dall'Aquilone. Chi si intendiam...


In [26]:
def create_instruction(sentence_to_translate, response, golden_label):
    instruction = f"The user needs to translate sentences from arcaic italian to modern italian. He provide this sentence to translate: {sentence_to_translate}",
    response = f"{response}"
    reference_answer = f"{golden_label}",
    
    rubric_data = {
      "criteria":"Is the model proficient in traqnslating archaic Italian sentences into modern Italian?",
      "score1_description":"The model isn't capable of translating archaic Italian sentences into modern Italian, producing responses that are either completely incorrect or irrelevant.",
      "score2_description":"The model is capable of translating some archaic Italian sentences into modern Italian, but the translations are often inaccurate or incomplete, failing to capture the essence of the original text.",
      "score3_description":"The model typically translates archaic Italian sentences into modern Italian with moderate accuracy, but may struggle with complex phrases or idiomatic expressions, leading to occasional inaccuracies.",
      "score4_description":"The model consistently translates archaic Italian sentences into modern Italian with high accuracy, capturing the essence of the original text and handling most idiomatic expressions effectively.",
      "score5_description":"The model excels in translating archaic Italian sentences into modern Italian, demonstrating a deep understanding of both the source and target languages, and consistently producing translations that are not only accurate but also stylistically appropriate."
    }
    
    score_rubric = SCORE_RUBRIC_TEMPLATE.format(**rubric_data)
    
    ABS_SYSTEM_PROMPT = "You are a helpful assistant that grades the quality of responses to user instructions. You will be given an instruction, a response, and a rubric. Your task is to assign a score based on the rubric. Your task is to assign a score based on the rubric in the following format 'Score:'."
    ABSOLUTE_PROMPT = f"Instruction: {{instruction}}\n\nResponse: {{response}}\n\nRubric: {{rubric}}\n\nReference Answer: {{reference_answer}}\n\nFeedback:"
    user_content = ABSOLUTE_PROMPT.format(
        instruction=instruction,
        response=response,
        rubric=score_rubric,
        reference_answer=reference_answer[0]
    )
    
    messages = [
        {"role": "system", "content": ABS_SYSTEM_PROMPT},
        {"role": "user", "content": user_content},
    ]

    return messages

def get_most_frequent_by_index(data):
  return [max(set(column), key=column.count) for column in zip(*data)]

def extract_score(text):
    """
    Extract score from feedback text using regex
    """
    # Primary pattern - looks for "score of [number]" after "Feedback:"
    pattern1 = r'(?i)(?:feedback:.*?)score\s+of\s+(\d+)'
    
    # Alternative pattern - more general "score [anything] [number]"
    pattern2 = r'(?i)(?:feedback:.*?)score\s+.*?\s+(\d+)'

    pattern3 = r'(?i)(?:feedback:.*?)score\b\s*:?\s*.*?(\d+)'
    # Try the more specific pattern first
    match = re.search(pattern1, text, re.DOTALL)
    if match:
        return int(match.group(1))
    
    # If that doesn't work, try the more general pattern
    match = re.search(pattern2, text, re.DOTALL)
    if match:
        return int(match.group(1))

    match = re.search(pattern3, text, re.DOTALL)
    if match:
        return int(match.group(1))
    
    return None

In [ ]:
length = len(dataset_golden)
list_of_messages_m = []
list_of_messages_d = []
for i in range(0, length):
    sentence_to_translate = dataset_golden["Sentence"][i]
    response_mistral = dataset_model_mistral["Mistral 7b-instruction"][i]
    response_deepseek = dataset_model_deepseek["Deepseek R1 qwen-8b"][i]
    golden_label = dataset_golden["goldenLabel"][i]
    message_mistral = create_instruction(sentence_to_translate, response_mistral, golden_label)
    message_deepseek = create_instruction(sentence_to_translate, response_deepseek, golden_label)
    list_of_messages_m.append(message_mistral)
    list_of_messages_d.append(message_deepseek)

### Evaluating Mistral

In [ ]:
rounds = 3

all_outputs_m = []
all_scores_m = []

print("Starting evaluation for Mistral 7B-Instruct")
for i in range(rounds):
    print(f"\n---- Round n. {i}----\n")
    list_of_scores_for_round = []
    list_of_outputs_for_round = []
    for msg in tqdm(list_of_messages_m, desc="Processing data"):
        encodeds = tokenizer.apply_chat_template(msg, return_tensors="pt", return_dict = True)
        model_inputs = encodeds['input_ids'].to(device)
        attention_mask = encodeds['attention_mask'].to(device)
        model.to(device)
        
        generated_ids = model.generate(model_inputs, max_new_tokens=1000, attention_mask=attention_mask, do_sample=True)
        decoded = tokenizer.batch_decode(generated_ids)
        list_of_outputs_for_round.append(decoded[0])
        output = extract_score(decoded[0])
        list_of_scores_for_round.append(output)
    all_outputs_m.append(list_of_outputs_for_round)
    all_scores_m.append(list_of_scores_for_round)

final_scores_m = get_most_frequent_by_index(all_scores_m)


---- Round n. 0----



Processing data: 100%|██████████| 97/97 [07:27<00:00,  4.61s/it]



---- Round n. 1----



Processing data: 100%|██████████| 97/97 [06:49<00:00,  4.23s/it]



---- Round n. 2----



Processing data: 100%|██████████| 97/97 [07:29<00:00,  4.63s/it]


### Evaluating Deepseek

In [ ]:
rounds = 3

all_outputs_d = []
all_scores_d = []

print("Starting evaluation for Deepseek R1 Qwen-8B")
for i in range(rounds):
    print(f"\n---- Round n. {i}----\n")
    list_of_scores_for_round = []
    list_of_outputs_for_round = []
    for msg in tqdm(list_of_messages_d, desc="Processing data"):
        encodeds = tokenizer.apply_chat_template(msg, return_tensors="pt", return_dict = True)
        model_inputs = encodeds['input_ids'].to(device)
        attention_mask = encodeds['attention_mask'].to(device)
        model.to(device)
        
        generated_ids = model.generate(model_inputs, max_new_tokens=1000, attention_mask=attention_mask, do_sample=True)
        decoded = tokenizer.batch_decode(generated_ids)
        list_of_outputs_for_round.append(decoded[0])
        output = extract_score(decoded[0])
        list_of_scores_for_round.append(output)
    all_outputs_d.append(list_of_outputs_for_round)
    all_scores_d.append(list_of_scores_for_round)
    
final_scores_d = get_most_frequent_by_index(all_scores_d)

### Cleaning cache

In [29]:
import torch
del tokenizer
del model
torch.cuda.empty_cache()

### Save results

#### Mistral

In [ ]:
dataset_evaluated_mistral = dataset_model_mistral.copy()
dataset_evaluated_mistral['p_mistral_vote'] = final_scores_m
dataset_evaluated_mistral['golden_label'] = dataset_golden["goldenLabel"]
dataset_evaluated_mistral.info()
# Save the evaluated dataset
dataset_evaluated_mistral.to_csv('./dataset/dataset_evaluated_mistral.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Author                  97 non-null     object 
 1   Date                    97 non-null     object 
 2   Region                  97 non-null     object 
 3   Sentence                97 non-null     object 
 4   Mistral 7b-instruction  97 non-null     object 
 5   prometeus vote          96 non-null     float64
 6   golden_label            97 non-null     object 
dtypes: float64(1), object(6)
memory usage: 5.4+ KB


#### Deepseek

In [ ]:
dataset_evaluated_deepseek = dataset_model_deepseek.copy()
dataset_evaluated_deepseek['p_mistral_vote'] = final_scores_d
dataset_evaluated_deepseek['golden_label'] = dataset_golden["goldenLabel"]
dataset_evaluated_deepseek.info()
# Save the evaluated dataset
dataset_evaluated_deepseek.to_csv('./dataset/dataset_evaluated_deepseek.csv', index=False)

## Metrics

In [33]:
import numpy as np
from sklearn.metrics import cohen_kappa_score
import pandas as pd
from scipy.stats import spearmanr

manual = pd.read_csv('./dataset/dataset_sub_20_m_eval.csv')
mistral = pd.read_csv('./dataset/dataset_evaluated_mistral.csv')
deepseek = pd.read_csv('./dataset/dataset_evaluated_deepseek.csv')

# select only the same sentence present in both mistral/deepseek and manual
mistral_sub = mistral[mistral['golden_label'].isin(manual['goldenLabel'])]
deepseek_sub = deepseek[deepseek['golden_label'].isin(manual['goldenLabel'])]

# order the mistral_sub, deepseek_sub and manual by golden_label
mistral_sub = mistral_sub.sort_values(by='golden_label').reset_index(drop=True)
deepseek_sub = deepseek_sub.sort_values(by='golden_label').reset_index(drop=True)
manual = manual.sort_values(by='goldenLabel').reset_index(drop=True)

# get the scores
mistral_scores = mistral_sub['p_mistral_vote'].values
deepseek_scores = deepseek_sub['p_deepseek_vote'].values
manual_scores_d = manual['user_eval_deepseek'].values
manual_scores_m = manual['user_eval_mistral'].values

In [34]:
# Calculate Cohen's Kappa for Mistral
kappa_mistral = cohen_kappa_score(manual_scores_m, mistral_scores)
# Calculate Cohen's Kappa for Deepseek
kappa_deepseek = cohen_kappa_score(manual_scores_d, deepseek_scores)
# Calculate Spearman's correlation for Mistral
spearman_mistral, _ = spearmanr(manual_scores_m, mistral_scores)
# Calculate Spearman's correlation for Deepseek
spearman_deepseek, _ = spearmanr(manual_scores_d, deepseek_scores)
# Print the results
print(f"Cohen's Kappa for Mistral: {kappa_mistral:.4f}")
print(f"Cohen's Kappa for Deepseek: {kappa_deepseek:.4f}")
print(f"Spearman's correlation for Mistral: {spearman_mistral:.4f}")
print(f"Spearman's correlation for Deepseek: {spearman_deepseek:.4f}")

Cohen's Kappa for Mistral: 0.2086
Cohen's Kappa for Deepseek: -0.0667
Spearman's correlation for Mistral: 0.1701
Spearman's correlation for Deepseek: 0.1345


In [36]:
# calculate the MSE for mistral and deepseek
from sklearn.metrics import mean_squared_error
mse_mistral = mean_squared_error(manual_scores_m, mistral_scores)
mse_deepseek = mean_squared_error(manual_scores_d, deepseek_scores)
print(f"Mean Squared Error for Mistral: {mse_mistral:.4f}")
print(f"Mean Squared Error for Deepseek: {mse_deepseek:.4f}")

Mean Squared Error for Mistral: 2.6500
Mean Squared Error for Deepseek: 2.3500
